<h1> Andrey Lukyanenko NER talk (sberloga) adption for ru WikiNer dataset

In [87]:
import sklearn_crfsuite
from sklearn_crfsuite import metrics
from sklearn_crfsuite import scorers
from sklearn.model_selection import train_test_split

import pandas as pd
import re

import bz2
from collections import Counter
# from operator import itemgetter


In [2]:
with bz2.open('../aij-wikiner-ru-wp3.bz2', 'rt', encoding='utf-8') as f:
    dataset = f.read()

In [34]:
dataset[:500]

'\nНа|PR|O севере|S|O граничит|V|O с|PR|O Латвией|S|I-LOC ,|PUNCT|O на|PR|O востоке|S|O --|PUNCT|O с|PR|O Белоруссией|S|I-LOC ,|PUNCT|O на|PR|O юго-западе|S|O --|PUNCT|O c|PR|O Польшей|S|I-LOC и|CONJ|O Калининградской|A|I-LOC областью|S|I-LOC России|S|B-LOC .|SENT|O\nВ|PR|O договоре|S|O среди|PR|O 5|NUM|O старших|A|O князей|S|O упоминается|V|O Миндовг|S|I-PER .|SENT|O\nОни|S-PRO|O покорили|V|O Пруссию|S|I-LOC и|CONJ|O Ливонию|S|I-LOC .|SENT|O\nВ|PR|O 1904|NUM|O году|S|O Запрет|S|I-MISC на|PR|I-MISC л'

In [35]:
sentences = dataset.split('\n')

In [62]:
data = []
for sent in sentences[1:]:
#     tokens = [itemgetter(*[0,2])(token.split('|')) if len(token)>2 else [] for token in sent.split()]    
    tokens = [tuple(token.split('|')) for token in sent.split()]
    data.append(tokens)

In [90]:
data[:10]

[[('На', 'PR', 'O'),
  ('севере', 'S', 'O'),
  ('граничит', 'V', 'O'),
  ('с', 'PR', 'O'),
  ('Латвией', 'S', 'I-LOC'),
  (',', 'PUNCT', 'O'),
  ('на', 'PR', 'O'),
  ('востоке', 'S', 'O'),
  ('--', 'PUNCT', 'O'),
  ('с', 'PR', 'O'),
  ('Белоруссией', 'S', 'I-LOC'),
  (',', 'PUNCT', 'O'),
  ('на', 'PR', 'O'),
  ('юго-западе', 'S', 'O'),
  ('--', 'PUNCT', 'O'),
  ('c', 'PR', 'O'),
  ('Польшей', 'S', 'I-LOC'),
  ('и', 'CONJ', 'O'),
  ('Калининградской', 'A', 'I-LOC'),
  ('областью', 'S', 'I-LOC'),
  ('России', 'S', 'B-LOC'),
  ('.', 'SENT', 'O')],
 [('В', 'PR', 'O'),
  ('договоре', 'S', 'O'),
  ('среди', 'PR', 'O'),
  ('5', 'NUM', 'O'),
  ('старших', 'A', 'O'),
  ('князей', 'S', 'O'),
  ('упоминается', 'V', 'O'),
  ('Миндовг', 'S', 'I-PER'),
  ('.', 'SENT', 'O')],
 [('Они', 'S-PRO', 'O'),
  ('покорили', 'V', 'O'),
  ('Пруссию', 'S', 'I-LOC'),
  ('и', 'CONJ', 'O'),
  ('Ливонию', 'S', 'I-LOC'),
  ('.', 'SENT', 'O')],
 [('В', 'PR', 'O'),
  ('1904', 'NUM', 'O'),
  ('году', 'S', 'O'),
  ('Запр

In [47]:
# df = pd.DataFrame([t for j in data for t in j], columns=['data', 'ents'])
# df['sent_id'] = [i for j in [[i] * len(s) for i, s in enumerate(data)] for i in j]

In [69]:
def word2features(sent, i):
    word = sent[i][0]
    postag = sent[i][1]
    
    features = {
        'bias': 1.0,
        'word.lower()': word.lower(),
        'word[-3:]': word[-3:],
        'word.isupper()': word.isupper(),
        'word.istitle()': word.istitle(),
        'word.isdigit()': word.isdigit(),
        'postag': postag,
    }
    if i > 0:
        word1 = sent[i-1][0]
        postag1 = sent[i-1][1]
        features.update({
            '-1:word.lower()': word1.lower(),
            '-1:word.istitle()': word1.istitle(),
            '-1:word.isupper()': word1.isupper(),
            '-1:postag': postag1,            
        })
    else:
        features['BOS'] = True
    
    if i < len(sent) - 1:
        word1 = sent[i+1][0]
        postag1 = sent[i+1][1]
        features.update({
            '+1:word.lower()': word1.lower(),
            '+1:word.istitle()': word1.istitle(),
            '+1:word.isupper()': word1.isupper(),
            '+1:postag': postag1,
        })
    else:
        features['EOS'] = True
        
    return features


def sent2features(sent):
    return [word2features(sent, i) for i in range(len(sent))]

def sent2labels(sent):
    return [label for token, postag, label in sent]

def sent2tokens(sent):
    return [token for token, postag, label in sent]   

In [70]:
X = [sent2features(s) for s in data]
y = [sent2labels(s) for s in data]

In [71]:
X[0][:2]

[{'bias': 1.0,
  'word.lower()': 'на',
  'word[-3:]': 'На',
  'word.isupper()': False,
  'word.istitle()': True,
  'word.isdigit()': False,
  'postag': 'PR',
  'BOS': True,
  '+1:word.lower()': 'севере',
  '+1:word.istitle()': False,
  '+1:word.isupper()': False,
  '+1:postag': 'S'},
 {'bias': 1.0,
  'word.lower()': 'севере',
  'word[-3:]': 'ере',
  'word.isupper()': False,
  'word.istitle()': False,
  'word.isdigit()': False,
  'postag': 'S',
  '-1:word.lower()': 'на',
  '-1:word.istitle()': True,
  '-1:word.isupper()': False,
  '-1:postag': 'PR',
  '+1:word.lower()': 'граничит',
  '+1:word.istitle()': False,
  '+1:word.isupper()': False,
  '+1:postag': 'V'}]

In [76]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

In [77]:
%%time
crf = sklearn_crfsuite.CRF(
    algorithm='lbfgs',
    c1=0.1,
    c2=0.1,
    max_iterations=100,
    all_possible_transitions=True,
    verbose=True,
)

crf.fit(X_train, y_train)

loading training data to CRFsuite: 100%|█████████████████████████████████████| 221182/221182 [00:22<00:00, 9847.38it/s]



Feature generation
type: CRF1d
feature.minfreq: 0.000000
feature.possible_states: 0
feature.possible_transitions: 1
0....1....2....3....4....5....6....7....8....9....10
Number of features: 720833
Seconds required: 5.698

L-BFGS optimization
c1: 0.100000
c2: 0.100000
num_memories: 6
max_iterations: 100
epsilon: 0.000010
stop: 10
delta: 0.000010
linesearch: MoreThuente
linesearch.max_iterations: 20

Iter 1   time=3.79  loss=2653553.60 active=718797 feature_norm=1.00
Iter 2   time=1.89  loss=2521037.73 active=703864 feature_norm=3.60
Iter 3   time=2.00  loss=1940386.22 active=698166 feature_norm=3.06
Iter 4   time=7.90  loss=1268259.01 active=702629 feature_norm=2.59
Iter 5   time=1.90  loss=1122937.20 active=712753 feature_norm=3.10
Iter 6   time=1.92  loss=849653.03 active=713717 feature_norm=5.18
Iter 7   time=1.92  loss=776750.40 active=691562 feature_norm=6.12
Iter 8   time=1.96  loss=712418.08 active=675785 feature_norm=6.70
Iter 9   time=2.04  loss=606399.02 active=601668 feature_

C:\Users\Asus\anaconda3\envs\nlp_env\lib\site-packages\sklearn\base.py:209: FutureWarning: From version 0.24, get_params will raise an AttributeError if a parameter cannot be retrieved as an instance attribute. Previously it would return None.
  warnings.warn('From version 0.24, get_params will raise an '


CRF(algorithm='lbfgs', all_possible_transitions=True, c1=0.1, c2=0.1,
    keep_tempfiles=None, max_iterations=100, verbose=True)

In [80]:
all_ents = list(crf.classes_)
ents = [ent for ent in all_ents if ent != 'O']

In [79]:
y_pred = crf.predict(X_test)
metrics.flat_f1_score(y_test, y_pred, average='weighted', labels=all_ents)

0.9743895300067786

In [81]:
y_pred = crf.predict(X_test)
metrics.flat_f1_score(y_test, y_pred, average='weighted', labels=ents)

0.8884256752687556

In [85]:
# group B and I results
sorted_ents = sorted(
    ents,
    key=lambda name: (name[1:], name[0])
)
print(metrics.flat_classification_report(
    y_test, y_pred, labels=sorted_ents, digits=3, zero_division=0
))

              precision    recall  f1-score   support

       B-LOC      0.832     0.677     0.747       483
       I-LOC      0.919     0.923     0.921     63587
      B-MISC      0.893     0.263     0.407        95
      I-MISC      0.869     0.773     0.819     49182
       B-ORG      0.000     0.000     0.000        17
       I-ORG      0.880     0.809     0.843     20955
       B-PER      0.000     0.000     0.000        50
       I-PER      0.948     0.939     0.944     45275

   micro avg      0.909     0.872     0.890    179644
   macro avg      0.668     0.548     0.585    179644
weighted avg      0.907     0.872     0.888    179644



In [88]:
def print_transitions(trans_features):
    for (label_from, label_to), weight in trans_features:
        print("%-6s -> %-7s %0.6f" % (label_from, label_to, weight))

print("Top likely transitions:")
print_transitions(Counter(crf.transition_features_).most_common(20))

print("\nTop unlikely transitions:")
print_transitions(Counter(crf.transition_features_).most_common()[-20:])

Top likely transitions:
I-MISC -> I-MISC  5.357717
I-LOC  -> B-LOC   4.743796
I-LOC  -> I-LOC   4.725043
I-PER  -> B-PER   4.533795
I-ORG  -> B-ORG   4.428066
I-ORG  -> I-ORG   4.355276
I-PER  -> I-PER   4.259028
B-MISC -> B-MISC  3.795842
I-MISC -> B-MISC  3.673432
O      -> O       2.626591
B-MISC -> I-MISC  1.931526
B-PER  -> I-PER   1.792172
B-LOC  -> I-LOC   1.784471
B-ORG  -> I-ORG   1.107734
I-MISC -> O       0.765725
O      -> I-MISC  0.685723
O      -> I-LOC   0.628833
B-LOC  -> B-LOC   0.577431
O      -> I-PER   0.470533
B-ORG  -> B-PER   -0.002863

Top unlikely transitions:
B-MISC -> I-PER   -2.698492
B-MISC -> I-ORG   -2.706838
I-MISC -> I-LOC   -2.765926
B-LOC  -> I-PER   -2.775726
B-PER  -> I-MISC  -2.838762
I-ORG  -> B-MISC  -2.913570
I-PER  -> I-LOC   -2.966066
I-ORG  -> I-LOC   -3.110156
I-PER  -> I-ORG   -3.202826
B-ORG  -> O       -3.284796
O      -> B-ORG   -3.419635
B-LOC  -> I-MISC  -3.576792
I-MISC -> B-LOC   -3.596053
B-LOC  -> I-ORG   -3.674142
I-ORG  -> B-LOC 